In [ ]:
try:
    import google.colab
    # We're in Colab
    !pip install git+https://github.com/landoskape/syd.git

except ImportError:
    # Not in Colab
    print("This notebook is designed for Google Colab.")
    print("To install locally, run: pip install syd")

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from syd import Viewer

class SimpleWaveformViewer(Viewer):
    """A simple example viewer that shows an interactive waveform."""

    def __init__(self):
        # Add parameters
        self.add_float("frequency", value=1.0, min_value=0.1, max_value=5.0)
        self.add_float("sine_amplitude", value=1.0, min_value=0.1, max_value=2.0)
        self.add_float("square_amplitude", value=1.0, min_value=0.1, max_value=2.0)
        self.add_float("sawtooth_amplitude", value=1.0, min_value=0.1, max_value=2.0)
        self.add_selection("sine_color", value="red", options=["red", "blue", "green"])
        self.add_selection("square_color", value="blue", options=["red", "blue", "green"])
        self.add_selection("sawtooth_color", value="green", options=["red", "blue", "green"])
        self.add_multiple_selection("waveform_type", value=["sine", "square", "sawtooth"], options=["sine", "square", "sawtooth"])
        self.add_boolean("show_legend", value=True)
        self.add_boolean("show_grid", value=True)

        self.add_button("randomize_colors", label="Randomize Colors", callback=self.randomize_colors)
        
        
        self.on_change("show_legend", self.update_grid)
        self.on_change("show_grid", self.update_legend)

    def update_grid(self, state):
        showing_details = state["show_legend"]
        self.update_boolean("show_grid", value=showing_details)

    def update_legend(self, state):
        showing_details = state["show_grid"]
        self.update_boolean("show_legend", value=showing_details)

    def randomize_colors(self, state):
        """Update the waveform type based on the frequency."""
        colors = ["red", "blue", "green"]
        random.shuffle(colors)
        self.update_selection("sine_color", value=colors[0])
        self.update_selection("square_color", value=colors[1])
        self.update_selection("sawtooth_color", value=colors[2])

    def plot(self, state):
        """Plot the waveform based on current parameters."""
        t = np.linspace(0, 2 * np.pi, 1000)

        ymin = float("inf")
        ymax = float("-inf")

        fig, ax = plt.subplots()
        if "sine" in state["waveform_type"]:
            ax.plot(
                t,
                state["sine_amplitude"] * np.sin(state["frequency"] * t),
                color=state["sine_color"],
                label="Sine",
            )
            ymin = min(ymin, -state["sine_amplitude"])
            ymax = max(ymax, state["sine_amplitude"])
        if "square" in state["waveform_type"]:
            ax.plot(
                t,
                state["square_amplitude"] * np.sign(np.sin(state["frequency"] * t)),
                color=state["square_color"],
                label="Square",
            )
            ymin = min(ymin, -state["square_amplitude"])
            ymax = max(ymax, state["square_amplitude"])
        if "sawtooth" in state["waveform_type"]:
            ax.plot(
                t,
                state["sawtooth_amplitude"]
                * (t % (2 * np.pi / state["frequency"]))
                * (state["frequency"] / 2 / np.pi),
                color=state["sawtooth_color"],
                label="Sawtooth",
            )
            ymin = min(ymin, -state["sawtooth_amplitude"])
            ymax = max(ymax, state["sawtooth_amplitude"])

        ax.set_xlabel("Time")
        ax.set_ylabel("Amplitude")
        ax.grid(state["show_grid"])
        ax.set_ylim(ymin * 1.1, ymax * 1.1)
        if state["show_legend"]:
            ax.legend()
        return fig
    
viewer = SimpleWaveformViewer()
_ = viewer.deploy(continuous=True)